<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Checks" data-toc-modified-id="Checks-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Checks</a></span></li></ul></li><li><span><a href="#Merging-&amp;-pruning-tree" data-toc-modified-id="Merging-&amp;-pruning-tree-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Merging &amp; pruning tree</a></span><ul class="toc-item"><li><span><a href="#Checks" data-toc-modified-id="Checks-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Checks</a></span></li></ul></li><li><span><a href="#Writing-tree" data-toc-modified-id="Writing-tree-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Writing tree</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Create phylogeny for all genome reps used for the Struo2 database
  * merging & filtering GTDB MLSA phylogenies

# Var

In [1]:
work_dir = '/ebio/abt3_projects2/databases_no-backup/GTDB/release202/Struo2/phylogeny/'

# species-rep genomes selected
genomes_file = file.path(dirname(work_dir),'metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv')

# trees from gtdb
arc_tree_file = '/ebio/abt3_projects2/databases_no-backup/GTDB/release202/phylogeny/ar122_r202.tree'
bac_tree_file = '/ebio/abt3_projects2/databases_no-backup/GTDB/release202/phylogeny/bac120_r202.tree'

# full gtdb metadata
gtdb_meta_dir = '/ebio/abt3_projects2/databases_no-backup/GTDB/release202/metadata/'
gtdb_meta_arc_file = file.path(gtdb_meta_dir, 'ar122_metadata_r202.tsv')
gtdb_meta_bac_file = file.path(gtdb_meta_dir, 'bac120_metadata_r202.tsv')

# Init

In [3]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(ape)
library(LeyLabRMisc)

In [4]:
df.dims()

# Load

In [5]:
tax_levs = c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species')

In [6]:
# genomes used for struo
genomes = Fread(genomes_file) %>%
    select.(ncbi_organism_name, accession, gtdb_taxonomy) %>%
    separate.(gtdb_taxonomy, tax_levs, sep = ';') %>%
    mutate.(Species = gsub(' ', '_', Species))
genomes %>% unique_n('genomes', ncbi_organism_name)
genomes

No. of unique genomes: 46434 


ncbi_organism_name,accession,Domain,Phylum,Class,Order,Family,Genus,Species
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
RS_GCF_000970205.1_Methanosarcina mazei S-6,RS_GCF_000970205.1,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanosarcina,s__Methanosarcina_mazei
GB_GCA_002499445.1_Methanosarcina sp. UBA293,GB_GCA_002499445.1,d__Archaea,p__Halobacteriota,c__Methanosarcinia,o__Methanosarcinales,f__Methanosarcinaceae,g__Methanosarcina,s__Methanosarcina_sp002499445
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
RS_GCF_013359645.1_Pseudomonas brassicacearum,RS_GCF_013359645.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E_kilonensis_B
RS_GCF_902810635.1_Chloroflexi bacterium,RS_GCF_902810635.1,d__Bacteria,p__Verrucomicrobiota,c__Verrucomicrobiae,o__Methylacidiphilales,f__Methylacidiphilaceae,g__CADDYQ01,s__CADDYQ01_sp902810725


In [7]:
# arc tree
arc_tree = read.tree(arc_tree_file)
arc_tree


Phylogenetic tree with 2339 tips and 2338 internal nodes.

Tip labels:
  RS_GCF_002286985.1, RS_GCF_004114995.1, RS_GCF_000337915.1, RS_GCF_007671685.1, RS_GCF_002727125.1, RS_GCF_000739595.1, ...
Node labels:
  d__Archaea, 100.0, 71.0, 50.0, 50.0, 79.0, ...

Rooted; includes branch lengths.

In [8]:
# bac tree
bac_tree = read.tree(bac_tree_file)
bac_tree


Phylogenetic tree with 45555 tips and 45554 internal nodes.

Tip labels:
  RS_GCF_001020875.1, RS_GCF_902498005.1, RS_GCF_014472415.1, RS_GCF_002813455.1, RS_GCF_009764645.1, RS_GCF_000282515.1, ...
Node labels:
  d__Bacteria, 65.0, 100.0, 10.0, 10.0, 1.0, ...

Rooted; includes branch lengths.

In [9]:
# metadata: archaea
gtdb_meta_arc = Fread(gtdb_meta_arc_file) %>%
    select.(accession, gtdb_taxonomy) %>%
    filter.(accession %in% arc_tree$tip.label) %>%
    separate.(gtdb_taxonomy, tax_levs, sep = ';')
gtdb_meta_arc

accession,Domain,Phylum,Class,Order,Family,Genus,Species
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GB_GCA_000200715.1,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum
GB_GCA_000247545.1,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
RS_GCF_900109065.1,d__Archaea,p__Halobacteriota,c__Halobacteria,o__Halobacteriales,f__Haloferacaceae,g__Halohasta,s__Halohasta litchfieldiae
RS_GCF_900114585.1,d__Archaea,p__Methanobacteriota,c__Methanobacteria,o__Methanobacteriales,f__Methanobacteriaceae,g__Methanobrevibacter,s__Methanobrevibacter olleyae


In [10]:
# metadata: bacteria
gtdb_meta_bac = Fread(gtdb_meta_bac_file) %>%
    select.(accession, gtdb_taxonomy) %>%
    filter.(accession %in% bac_tree$tip.label) %>%
    separate.(gtdb_taxonomy, tax_levs, sep = ';')
gtdb_meta_bac

accession,Domain,Phylum,Class,Order,Family,Genus,Species
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GB_GCA_000024525.1,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Cytophagales,f__Spirosomaceae,g__Spirosoma,s__Spirosoma linguale
GB_GCA_000285855.2,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Lachnospirales,f__Lachnospiraceae,g__Blautia_A,s__Blautia_A sp000285855
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
RS_GCF_903643055.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Rhizobacter,s__Rhizobacter sp903643055
RS_GCF_903645355.1,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Actinomycetales,f__Actinomycetaceae,g__Pauljensenia,s__Pauljensenia sp903645355


In [11]:
# combined
gtdb_meta = rbind(gtdb_meta_arc, gtdb_meta_bac)
gtdb_meta_arc = gtdb_meta_bac = NULL
gtdb_meta

accession,Domain,Phylum,Class,Order,Family,Genus,Species
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GB_GCA_000200715.1,d__Archaea,p__Thermoproteota,c__Nitrososphaeria,o__Nitrososphaerales,f__Nitrosopumilaceae,g__Cenarchaeum,s__Cenarchaeum symbiosum
GB_GCA_000247545.1,d__Archaea,p__Thermoproteota,c__Thermoproteia,o__Thermoproteales,f__Thermoproteaceae,g__Pyrobaculum,s__Pyrobaculum oguniense
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
RS_GCF_903643055.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Rhizobacter,s__Rhizobacter sp903643055
RS_GCF_903645355.1,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Actinomycetales,f__Actinomycetaceae,g__Pauljensenia,s__Pauljensenia sp903645355


## Checks

In [12]:
summary_x(arc_tree$edge.length)
summary_x(bac_tree$edge.length)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
edge.length,0,0.028705,0.06437,0.1287773,0.1504475,1.98851,0.784,0.32


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
edge.length,5e-04,0.01201,0.02894,0.0561111,0.0678525,1.9046,0.764,0.312


# Merging & pruning tree

In [13]:
# binding trees at root
tree = ape::bind.tree(arc_tree, bac_tree)
tree


Phylogenetic tree with 47894 tips and 47891 internal nodes.

Tip labels:
  RS_GCF_002286985.1, RS_GCF_004114995.1, RS_GCF_000337915.1, RS_GCF_007671685.1, RS_GCF_002727125.1, RS_GCF_000739595.1, ...
Node labels:
  d__Archaea, 100.0, 71.0, 50.0, 50.0, 79.0, ...

Unrooted; includes branch lengths.

In [14]:
# renaming as species
idx = gtdb_meta %>%
    filter.(accession %in% tree$tip.label) %>%
    select.(accession, Species) %>%
    mutate.(Species = gsub(' ', '_', Species)) %>%
    as.data.frame
rownames(idx) = idx$accession
tree$tip.label = idx[tree$tip.label,'Species']
tree


Phylogenetic tree with 47894 tips and 47891 internal nodes.

Tip labels:
  s__Halorubrum_sp002286985, s__Halorubrum_amylolyticum, s__Halorubrum_saccharovorum, s__Halorubrum_salsamenti, s__Halorubrum_persicum, s__Halorubrum_halophilum, ...
Node labels:
  d__Archaea, 100.0, 71.0, 50.0, 50.0, 79.0, ...

Unrooted; includes branch lengths.

In [15]:
# checking overlap
overlap(genomes$Species, tree$tip.label)

intersect(x,y): 46434 
setdiff(x,y): 0 
setdiff(y,x): 1460 
union(x,y): 47894 


In [16]:
# purning
to_rm = setdiff(tree$tip.label, genomes$Species)
to_rm %>% length
tree_f = ape::drop.tip(tree, to_rm)
tree_f

[1] 1460


Phylogenetic tree with 46434 tips and 46431 internal nodes.

Tip labels:
  s__Halorubrum_sp002286985, s__Halorubrum_amylolyticum, s__Halorubrum_saccharovorum, s__Halorubrum_salsamenti, s__Halorubrum_persicum, s__Halorubrum_halophilum, ...
Node labels:
  d__Archaea, 100.0, 71.0, 50.0, 50.0, 79.0, ...

Unrooted; includes branch lengths.

## Checks

In [20]:
# checking overlap
overlap(genomes$Species, tree_f$tip.label)

intersect(x,y): 46434 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 46434 


In [21]:
# branch lengths
summary_x(tree_f$edge.length)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
edge.length,0,0.01233,0.02995,0.05987137,0.07089,1.98851,0.798,0.326


# Writing tree

In [23]:
F = file.path(work_dir, 'ar122-bac120_r202_1per-GTDB-Spec_gte50comp-lt5cont.nwk')
write.tree(tree_f, F)
cat('File writen:', F, '\n')

File writen: /ebio/abt3_projects2/databases_no-backup/GTDB/release202/Struo2/phylogeny//ar122-bac120_r202_1per-GTDB-Spec_gte50comp-lt5cont.nwk 


# sessionInfo

In [24]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq2/lib/libopenblasp-r0.3.10.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.9 ape_5.4-1         tidytable_0.5.8   data.table_1.13.6
[5] ggplot2_3.3.3     tidyr_1.1.2       dplyr_1.0.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.6        pillar_1.4.7      compiler_4.0.3    base64enc_0.1-3  
 [5] tools_4.0.3       digest_0